### Предварительные сведения

Векторные нормы:

$||u||_{\infty} = \max_i|u_i|$

$||u||_1 = \sum_i |u_i|$

$||u||_2 = \left(\sum_i |u_i|^2 \right)^{\frac{1}{2}}$

Матричные нормы:

$||A||_{\infty} = \max_i \sum_j |a_{ij}|$

$||A||_1 = \max_j \sum_i |a_{ij}|$

$||A||_2 = \left(\max_i \lambda_i(A A^*) \right)^{\frac{1}{2}}$

Контрольный вопрос: какова будет вторая норма матрицы, если матрица самосопряженная?

*Ваш ответ на контрольный вопрос*: 
$$ ||A||_2 = \max_i |\lambda_i|$$
Для эрмитовой матрицы $A$ существует базис из собственнных векторов, в котором данная матрица будет диагональной (на диагонали собственные числа). Если матрица самосопряженная $A = A^*$, то $AA^* = A^2$ или в диагональном представлении - матрица, на диагонали которой квадраты собсвенных чисел. А значит, при извлечении корня квадратного из максимального квадрата $\lambda_i$ останется наше выражение.

In [115]:
import numpy as np
import numpy.linalg as la

A = np.array([[1,2],[3,4]])
v = range(0,3)
Vander = np.vander(v)
print('norm_1 = ', la.norm(Vander, 1))
print('norm_2 = ', la.norm(Vander, 2))
print('norm_inf = ', la.norm(Vander, np.inf))
Vander

norm_1 =  5.0
norm_2 =  4.844958524498339
norm_inf =  7.0


array([[0, 0, 1],
       [1, 1, 1],
       [4, 2, 1]])

Обусловленность:
$$(A+\delta A)u = f + \delta f$$
$$\frac{||\delta u||}{||u||}\le \frac{\mu}{1-\mu\frac{||\delta A||}{||A||}} \left(\frac{||\delta f||}{||f||}+\frac{||\delta A||}{||A||}\right)$$

$\mu$ - число обусловленности матрицы A, $\mu(A) = ||A^{-1}||\cdot||A||$, $\mu \ge 1$.



## Пример проблемы использования метода Гаусса для решения СЛАУ

In [116]:
import numpy as np
import numpy.linalg as la

def gauss( A, b ):
    n = b.size
    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i,k]!=0:
                c = A[i,k]/A[k,k]
                A[i,k+1:n] = A[i,k+1:n] - c*A[k,k+1:n]
                b[i] = b[i] - c*b[k]
                
    # обратный ход
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(A[k,k+1:n],b[k+1:n]))/A[k,k];
    return b

#все числа в представлены как вещественные
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

In [117]:
print(A1)

[[ 1.e-16  1.e+00 -1.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 2.e+00 -1.e+00  0.e+00]]


In [118]:
print(A2)

[[ 2.e+00 -1.e+00  0.e+00]
 [-1.e+00  2.e+00 -1.e+00]
 [ 1.e-16  1.e+00 -1.e+00]]


In [119]:
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu1 = ', la.cond(A1))
print('mu2 = ', la.cond(A2))

print('u1 = ', gauss(A1, b1))
#print('u1 = ', la.solve(A1, b1))
print('u2 = ', gauss(A2, b2))#la.solve(A2, b2))

mu1 =  16.39373162228438
mu2 =  16.393731622284395
u1 =  [0.55511151 0.25       0.25      ]
u2 =  [1. 1. 1.]


## Часть 1. LU разложение

Задание: 

реализовать алгоритм решения предыдущей задачи с матрицей A2 с помощью LU-разложение В решении должна выводиться L, U и собственно решение системы. 

ВАЖНО: реализация метода LU должна быть получена путем небольшой модификации метода gauss!  При это саму реализацию можно разделить на два метода: один метод собственно находит LU разложение (можно сделать переделкой цикла для матрицы A метода gauss), второй метод - непосредственное решение системы с помощью прямого и обратного хода. Ни в каком виде нельзя пользоваться пакетными методами (в частности, la.solve)

In [120]:
def LU( A, b ):
    n = b.size
    f = b.copy()
    #создаем матрицу под L
    L = np.zeros((n, n))
    for i in range(0,n):
        L[i,i] = 1
    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i,k]!=0:
                c = A[i,k]/A[k,k]
                A[i,k:n] = A[i,k:n] - c*A[k,k:n]
                b[i] = b[i] - c*b[k]
                L[i,k] = c
    #Решаем Lv = f, прямой ход (LUx = f <=> Lv = f, Ux = v)
    v = [0]*n
    for k in range(0, n):
        v[k] = f[k] - np.dot(L[k,], v)
    # Решаем Ux = v, обратный ход
    x = [0]*n
    for k in range(n-1,-1,-1):
        x[k] = (v[k] - np.dot(A[k,], x))/A[k,k] 
    
    return L, A, x
    

In [121]:
A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu2 = ', la.cond(A2))

L, U, x = LU(A2, b2)

print("L = \n", L)
print("U = \n", U)
print("x = ", x)

mu2 =  16.393731622284395
L = 
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.00000000e-01  1.00000000e+00  0.00000000e+00]
 [ 5.00000000e-17  6.66666667e-01  1.00000000e+00]]
U = 
 [[ 2.         -1.          0.        ]
 [ 0.          1.5        -1.        ]
 [ 0.          0.         -0.33333333]]
x =  [1.0, 1.0, 1.0]


### LU - разложение с помощью пакета sympy

Чтобы убедиться, что разложение получено верно, можно воспользоваться скриптом ниже

In [122]:
import sympy as sp

#A = sp.Matrix([[2, 3], [5, 4]])
A = sp.Matrix([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
L, U, _ = A.LUdecomposition()

In [123]:
L

Matrix([
[      1,                 0, 0],
[   -0.5,                 1, 0],
[5.0e-17, 0.666666666666667, 1]])

In [124]:
U

Matrix([
[2.0, -1.0,                0.0],
[  0,  1.5,               -1.0],
[  0,    0, -0.333333333333333]])

## Часть 2. Нахождение обратной матрицы с помощью LU разложения

Задание:

Предложить алгоритм с использованием LU-разложения и найти обратную матрицу с точностью $\epsilon = 10^{-3}$:
$$
A = \begin{pmatrix} 
1 & 1  & 1 \\
0 & 1 & 2 \\
7 & 1 & 4 \\
\end{pmatrix}
$$
Для необходимых оценок использовать первую норму. Сравнить результат со значением, найденным с помощью функции numpy.inv.

    Aлгоритм:
Находим $LU$-разложение. $A = LU$. Теперь решаем $n$ систем уравнений вида $LUa_i = E_i$, где $a_i$ и $E_i$ $i$-ые столбцы матриц $A^{-1}$ и единичной $E$ соответственно. Находим все столбцы и составляем ответ. Каждый столбец $a_i$ нахожу, как и в предыдущем пункте:
1. Решаю $Lv_i = E_i$, где $v_i = Ua_i$
2. Дальше решаем $Ua_i = v_i$
 Я буду пользоваться уже написанной функцией для $LU$-разложения.

In [125]:
#создаем, чтобы не засорять, дополнительно функции для прямого и обратного хода:
def straight(L, e):
    n = e.size
    v = [0]*n
    for k in range(0, n):
        v[k] = e[k] - np.dot(L[k,], v)
    return np.array(v)

def back(U, v):
    n = v.size
    x = [0]*n
    for k in range(n-1,-1,-1):
        x[k] = (v[k] - np.dot(A[k,], x))/A[k,k]
    return x


In [126]:
#A^{-1} обозначим A_1
def inverse_matrix(A):
    n = len(A)
    A_1 = np.zeros((n,n))
    L, U, _ = LU(A, np.array([0, 0, 0]))
    E = np.eye(3)
    for i in range(0, n):
        A_1[:, i] = back(U, straight(L, E[:, i]))
    return A_1

In [127]:
A = np.array([[1, 1, 1], [0, 1, 2], [7, 1, 4]])
print(inverse_matrix(A))

[[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]]


Проверим наш ответ

In [128]:
A = np.array([[1, 1, 1], [0, 1, 2], [7, 1, 4]])
la.inv(A)

array([[ 0.22222222, -0.33333333,  0.11111111],
       [ 1.55555556, -0.33333333, -0.22222222],
       [-0.77777778,  0.66666667,  0.11111111]])

Число обусловленности матрицы $\mu(A) = 8*\frac{23}{9}$